In [ ]:
### Fermentation Dynamic Model

In [1]:
import numpy as np
import pandas as pd
import math as m
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
from ipynb.fs.full.Setup import * 
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
tab_nest

In [3]:
interact_manual(simulation);

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…